# Predicting store sales using Simple Moving average

In [1]:
import pandas as pd

In [24]:
df = pd.read_excel('online_retail.xlsx')

In [26]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Starting EDA to have only the values we need

In [27]:
## First calculating the amount for the item 
df['price'] = (df['Quantity'] * df['UnitPrice'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [28]:
import datetime as datetime

In [29]:
df = df.rename(columns={'InvoiceDate':'Date'})

In [31]:
#taking only columns that we need
df = df[['Date','price']]
df.head()

,Date,price
0,2010-12-01 08:26:00,15.30
1,2010-12-01 08:26:00,20.34
2,2010-12-01 08:26:00,22.00
3,2010-12-01 08:26:00,20.34
4,2010-12-01 08:26:00,20.34


Lets look at whats the range of whole data and see if anything is missing

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    541909 non-null  datetime64[ns]
 1   price   541909 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.3 MB


all the values are non null.
lets check from which date to last date we have the data 

In [34]:
df.Date.min(),df.Date.max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

we have the data from dec 2010 to nov 2011 that is complete and incomplete data for dec 2011.
Lets train the model from dec 2010 to oct 2011 and check for nov 2011 then we will do testing with nov 2011 and final validation on dec 2011. 

Final Summary - 
1. Training -> dec 2010 to nov 2011
2. Testing -> nov 2011
3. Validation -> dec 2011

In [23]:
df_final = df.set_index('date')

<class 'pandas.core.frame.DataFrame'>
Index: 541909 entries, 2010-12-01 to 2011-12-09
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   price   541909 non-null  float64
dtypes: float64(1)
memory usage: 8.3+ MB


In [21]:
df_final = df_final.resample('M')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'